# G4 - Anomaly Detection Analysis
## Exploratory Data Analysis and Model Evaluation

**Project**: SDID 2025/2026  
**Group**: G4 - Anomaly Detection & ROI

In [ ]:
# Imports
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from src.database import DatabaseConnection
from src.preprocessor import DataPreprocessor
from src.anomaly_detector import AnomalyDetector
from src.roi_calculator import ROICalculator

# Plotting configuration
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

## 1. Data Loading

In [ ]:
# Connect to database
db = DatabaseConnection()
db.connect()

# Load data
df = db.get_historical_data(limit=10000)
print(f"Loaded {len(df)} records")
df.head()

## 2. Data Exploration

In [ ]:
# Basic statistics
df.describe()

In [ ]:
# Distribution of key features
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

axes[0, 0].hist(df['global_active_power'], bins=50, alpha=0.7)
axes[0, 0].set_title('Global Active Power Distribution')
axes[0, 0].set_xlabel('Power (kW)')

axes[0, 1].hist(df['voltage'], bins=50, alpha=0.7, color='orange')
axes[0, 1].set_title('Voltage Distribution')
axes[0, 1].set_xlabel('Voltage (V)')

axes[1, 0].hist(df['global_intensity'], bins=50, alpha=0.7, color='green')
axes[1, 0].set_title('Global Intensity Distribution')
axes[1, 0].set_xlabel('Intensity (A)')

axes[1, 1].hist(df['global_reactive_power'], bins=50, alpha=0.7, color='red')
axes[1, 1].set_title('Global Reactive Power Distribution')
axes[1, 1].set_xlabel('Power (kVAR)')

plt.tight_layout()
plt.show()

## 3. Data Preprocessing (G3 Parameters)

In [ ]:
# Load G3 preprocessor
preprocessor = DataPreprocessor()

# Try to load G3 parameters
if not preprocessor.load_g3_parameters():
    print("G3 parameters not found, fitting on current data...")
    preprocessor.fit_default(df)

# Transform data
X_transformed = preprocessor.transform(df)
print(f"Transformed shape: {X_transformed.shape}")

# Feature importance
print("\nPCA Components:")
preprocessor.get_feature_importance()

In [ ]:
# Visualize PCA components
fig = plt.figure(figsize=(12, 4))

ax1 = fig.add_subplot(131)
ax1.scatter(X_transformed[:, 0], X_transformed[:, 1], alpha=0.3, s=1)
ax1.set_xlabel('PC1')
ax1.set_ylabel('PC2')
ax1.set_title('PC1 vs PC2')

ax2 = fig.add_subplot(132)
ax2.scatter(X_transformed[:, 0], X_transformed[:, 2], alpha=0.3, s=1)
ax2.set_xlabel('PC1')
ax2.set_ylabel('PC3')
ax2.set_title('PC1 vs PC3')

ax3 = fig.add_subplot(133)
ax3.scatter(X_transformed[:, 1], X_transformed[:, 2], alpha=0.3, s=1)
ax3.set_xlabel('PC2')
ax3.set_ylabel('PC3')
ax3.set_title('PC2 vs PC3')

plt.tight_layout()
plt.show()

## 4. Anomaly Detection Model

In [ ]:
# Train model
detector = AnomalyDetector(algorithm='isolation_forest')
detector.train(X_transformed)

# Predict on same data
scores, predictions = detector.predict(X_transformed)

print(f"Anomalies detected: {sum(predictions)}/{len(predictions)} ({sum(predictions)/len(predictions)*100:.2f}%)")

In [ ]:
# Score distribution
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(scores, bins=50, alpha=0.7, edgecolor='black')
plt.axvline(detector.threshold, color='red', linestyle='--', linewidth=2, label=f'Threshold: {detector.threshold}')
plt.xlabel('Anomaly Score')
plt.ylabel('Frequency')
plt.title('Distribution of Anomaly Scores')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.hist(scores[predictions], bins=30, alpha=0.7, color='red', label='Anomalies')
plt.hist(scores[~predictions], bins=30, alpha=0.7, color='blue', label='Normal')
plt.xlabel('Anomaly Score')
plt.ylabel('Frequency')
plt.title('Score Distribution by Class')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Visualize anomalies in PCA space
fig = plt.figure(figsize=(15, 5))

# PC1 vs PC2
ax1 = fig.add_subplot(131)
ax1.scatter(X_transformed[~predictions, 0], X_transformed[~predictions, 1], 
           alpha=0.3, s=10, c='blue', label='Normal')
ax1.scatter(X_transformed[predictions, 0], X_transformed[predictions, 1], 
           alpha=0.8, s=50, c='red', marker='x', label='Anomaly')
ax1.set_xlabel('PC1')
ax1.set_ylabel('PC2')
ax1.set_title('Anomalies in PC1-PC2 Space')
ax1.legend()
ax1.grid(True, alpha=0.3)

# PC1 vs PC3
ax2 = fig.add_subplot(132)
ax2.scatter(X_transformed[~predictions, 0], X_transformed[~predictions, 2], 
           alpha=0.3, s=10, c='blue', label='Normal')
ax2.scatter(X_transformed[predictions, 0], X_transformed[predictions, 2], 
           alpha=0.8, s=50, c='red', marker='x', label='Anomaly')
ax2.set_xlabel('PC1')
ax2.set_ylabel('PC3')
ax2.set_title('Anomalies in PC1-PC3 Space')
ax2.legend()
ax2.grid(True, alpha=0.3)

# PC2 vs PC3
ax3 = fig.add_subplot(133)
ax3.scatter(X_transformed[~predictions, 1], X_transformed[~predictions, 2], 
           alpha=0.3, s=10, c='blue', label='Normal')
ax3.scatter(X_transformed[predictions, 1], X_transformed[predictions, 2], 
           alpha=0.8, s=50, c='red', marker='x', label='Anomaly')
ax3.set_xlabel('PC2')
ax3.set_ylabel('PC3')
ax3.set_title('Anomalies in PC2-PC3 Space')
ax3.legend()
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Anomaly Analysis

In [ ]:
# Add predictions to dataframe
df['anomaly_score'] = scores
df['is_anomaly'] = predictions

# Compare normal vs anomalous data
print("Normal data statistics:")
print(df[~df['is_anomaly']][['global_active_power', 'voltage', 'global_intensity']].describe())

print("\nAnomalous data statistics:")
print(df[df['is_anomaly']][['global_active_power', 'voltage', 'global_intensity']].describe())

In [ ]:
# Top 10 anomalies
top_anomalies = df[df['is_anomaly']].nsmallest(10, 'anomaly_score')
print("Top 10 Most Anomalous Records:")
top_anomalies[['timestamp', 'global_active_power', 'voltage', 'anomaly_score']]

## 6. ROI Analysis

In [ ]:
# Calculate ROI
roi_calc = ROICalculator()

if roi_calc.connect():
    # Get complete data
    df_roi = roi_calc.get_anomaly_data()
    
    if len(df_roi) > 0:
        analysis = roi_calc.calculate_roi(df_roi, system_cost=10000)
        
        # Display results
        print("\n" + "="*70)
        print("ROI ANALYSIS SUMMARY")
        print("="*70)
        
        roi = analysis['roi_summary']
        print(f"System Cost:        ${roi['system_cost']:,.2f}")
        print(f"Total Benefits:     ${roi['total_benefits']:,.2f}")
        print(f"Net Benefit:        ${roi['net_benefit']:,.2f}")
        print(f"ROI:                {roi['roi_percentage']:.2f}%")
        print(f"Payback Period:     {roi['payback_period_days']:.0f} days")
        print("="*70)
    
    roi_calc.disconnect()

## 7. Threshold Sensitivity Analysis

In [ ]:
# Test different thresholds
thresholds = np.linspace(-1.0, 0.0, 50)
anomaly_rates = []

for thresh in thresholds:
    is_anom = scores < thresh
    anomaly_rates.append(sum(is_anom) / len(is_anom) * 100)

plt.figure(figsize=(10, 6))
plt.plot(thresholds, anomaly_rates, linewidth=2)
plt.axvline(detector.threshold, color='red', linestyle='--', 
           label=f'Current threshold: {detector.threshold}')
plt.axhline(1, color='green', linestyle=':', alpha=0.5, label='1% anomaly rate')
plt.axhline(5, color='orange', linestyle=':', alpha=0.5, label='5% anomaly rate')
plt.xlabel('Threshold')
plt.ylabel('Anomaly Rate (%)')
plt.title('Threshold Sensitivity Analysis')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 8. Export Results

In [ ]:
# Save model
detector.save_model('../models/anomaly_detector.pkl')
print("✓ Model saved")

# Save analysis results
results = {
    'total_records': len(df),
    'anomalies_detected': int(sum(predictions)),
    'anomaly_rate': float(sum(predictions)/len(predictions)*100),
    'threshold': float(detector.threshold),
    'score_mean': float(scores.mean()),
    'score_std': float(scores.std())
}

import json
with open('../docs/analysis_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("✓ Results saved to docs/analysis_results.json")

In [ ]:
# Cleanup
db.disconnect()
print("\n✓ Analysis complete")